IMPORT DATA AND READ



In [2]:
import pandas as pd
import numpy as np
from numpy.linalg import norm
import random
import datetime as dt
from sklearn.metrics.pairwise import cosine_similarity
import itertools

In [3]:
path = '/content/gdrive/MyDrive/'

from google.colab import drive
drive.mount('/content/gdrive')

image_emb = path + "article_dictionary.csv"

description_emb = path + "desctiption_embeddings.csv"

prod_network = path + "new_pn.csv"

article_path = path + "articles.csv"

item_count = path + "item_count.csv"

Mounted at /content/gdrive


In [4]:
articles = pd.read_csv(article_path)
articles.head()

,article_id,product_code,prod_name,product_type_no,product_type_name,product_group_name,graphical_appearance_no,graphical_appearance_name,colour_group_code,colour_group_name,...,department_name,index_code,index_name,index_group_no,index_group_name,section_no,section_name,garment_group_no,garment_group_name,detail_desc
0,108775015,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,9,Black,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
1,108775044,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,10,White,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
2,108775051,108775,Strap top (1),253,Vest top,Garment Upper body,1010017,Stripe,11,Off White,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
3,110065001,110065,OP T-shirt (Idro),306,Bra,Underwear,1010016,Solid,9,Black,...,Clean Lingerie,B,Lingeries/Tights,1,Ladieswear,61,Womens Lingerie,1017,"Under-, Nightwear","Microfibre T-shirt bra with underwired, moulde..."
4,110065002,110065,OP T-shirt (Idro),306,Bra,Underwear,1010016,Solid,10,White,...,Clean Lingerie,B,Lingeries/Tights,1,Ladieswear,61,Womens Lingerie,1017,"Under-, Nightwear","Microfibre T-shirt bra with underwired, moulde..."


In [5]:
description_emb = pd.read_csv(description_emb,index_col=False)
description_emb.drop(["description","Description_new"],axis=1,inplace=True)
description_emb.set_index('product_id', inplace=True)
description_emb.head()

,0,1,2,3,4,5,6,7,8,9,...,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023
product_id,,,,,,,,,,,,,,,,,,,,,
108775015,0.643704,-0.728313,-0.370916,0.930039,0.205670,-0.512119,-0.781456,0.846769,0.224285,0.927329,...,0.184097,-0.499265,0.154369,0.201446,-1.428334,1.050288,-0.188325,0.233521,-0.609104,0.525521
108775044,0.643703,-0.728313,-0.370917,0.930038,0.205671,-0.512120,-0.781456,0.846770,0.224286,0.927330,...,0.184097,-0.499264,0.154369,0.201446,-1.428334,1.050289,-0.188325,0.233522,-0.609104,0.525522
108775051,0.643703,-0.728313,-0.370917,0.930038,0.205671,-0.512120,-0.781456,0.846770,0.224286,0.927330,...,0.184097,-0.499264,0.154369,0.201446,-1.428334,1.050289,-0.188325,0.233522,-0.609104,0.525522
110065001,-0.688227,0.463673,-0.972770,0.495205,0.684199,0.604548,-0.995238,0.079121,-0.119942,1.173021,...,0.412209,-0.001696,-1.001149,1.065278,1.280330,1.151635,-1.661794,1.554372,-0.640086,1.515295
110065002,-0.688227,0.463673,-0.972770,0.495205,0.684199,0.604548,-0.995238,0.079121,-0.119942,1.173021,...,0.412209,-0.001696,-1.001149,1.065278,1.280330,1.151635,-1.661794,1.554372,-0.640086,1.515295


In [6]:
images_df = pd.read_csv(image_emb)
images_df = images_df.rename(columns={'Unnamed: 0':'product_id'})
images_df.set_index('product_id', inplace=True)
images_df = images_df[images_df.index.isin(description_emb.index)]
images_df.head()

,0,1,2,3,4,5,6,7,8,9,...,502,503,504,505,506,507,508,509,510,511
product_id,,,,,,,,,,,,,,,,,,,,,
663713001,0.615762,0.081262,0.516690,0.962563,0.924353,1.684056,0.687998,2.590135,1.345514,0.932243,...,0.722571,0.493880,1.514934,0.808707,0.175722,0.075635,2.540155,0.443045,0.144813,3.188583
541518023,1.309825,0.251596,2.650270,0.854556,0.499745,0.640864,0.317741,2.482037,1.334311,0.160331,...,0.805906,0.288806,1.909185,1.035806,0.235855,0.029039,1.755495,0.214506,0.068759,1.479373
505221004,0.643801,1.510989,0.000962,0.083988,0.042613,0.592353,0.222060,0.029552,3.803942,0.287578,...,4.138981,0.000378,1.651003,1.102987,0.376301,0.494068,0.162560,0.728679,0.592000,0.645665
685687003,0.416132,0.416597,0.004070,1.183370,0.039602,0.719203,0.099739,0.302293,2.071510,0.333582,...,0.368519,0.018202,1.945736,2.577750,0.341443,0.001426,0.894459,0.927478,0.049759,0.403430
685687004,0.665126,0.647309,0.020886,0.783940,0.089257,1.186417,0.576247,0.077178,2.789533,0.139908,...,0.337211,0.016081,1.795114,2.542928,0.192679,0.102081,0.774224,1.159667,0.022634,0.813263


In [7]:
network_df = pd.read_csv(prod_network,usecols=["a","b","count","cs"])
network_df = network_df[network_df["a"].isin(description_emb.index) & network_df["b"].isin(description_emb.index)]
#network_df = network_df.sort_values(by=["count","cs"],ascending=[False,False])
network_df.head()

,a,b,count,cs
0,684209013,688537011,10864,0.680656
1,689109001,692930001,8701,0.758178
2,689109003,692930003,8454,0.708184
3,706016001,706016002,8341,0.854595
4,501616007,507909001,8224,0.897532


In [8]:
item_count = pd.read_csv(item_count)
item_count.head()

,Unnamed: 0,article,count
0,0,108775015,67010
1,1,108775044,50771
2,2,108775051,2977
3,3,110065001,5094
4,4,110065002,3528


In [9]:
transactions_df = pd.read_csv(path + "transactions_train.csv")
transactions_df.head()

,t_dat,customer_id,article_id,price,sales_channel_id
0,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,663713001,0.050831,2
1,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,541518023,0.030492,2
2,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,505221004,0.015237,2
3,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687003,0.016932,2
4,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687004,0.016932,2


PREPARE THE DATA

In [10]:
# We created a list of values ​​for the embedding description to make sure that all the products that are there - will be the products that are in all the data
product_id_list = description_emb.index.tolist()
len(set(product_id_list))

103246

In [11]:
# We created a list of values ​​for the articles to make sure that all the products that are there - will be the products that are in all the data
# Because the embedding description has duplicates - we only took the products that do not have duplicates in the description
first_rows_per_product_code = articles.drop_duplicates(subset='detail_desc', keep='first')
product_id_list2 = first_rows_per_product_code.article_id.tolist()
len(set(product_id_list2))

43405

In [12]:
#We chose the 4 columns that we saw that have the meaning to be tabular - separately calculated to perform the feature selection
selected_columns = articles[['article_id', 'product_type_name', 'product_group_name', 'graphical_appearance_name', 'index_group_name']]

selected_columns.head()

,article_id,product_type_name,product_group_name,graphical_appearance_name,index_group_name
0,108775015,Vest top,Garment Upper body,Solid,Ladieswear
1,108775044,Vest top,Garment Upper body,Solid,Ladieswear
2,108775051,Vest top,Garment Upper body,Stripe,Ladieswear
3,110065001,Bra,Underwear,Solid,Ladieswear
4,110065002,Bra,Underwear,Solid,Ladieswear


In [13]:
# We made a threshold here on the product features

def replace_rare_categories(df, threshold_percentage=0.05):
    total_rows = len(df)
    for col in df.columns:
        if col == 'article_id':
            continue  # Skip processing the 'article_id' column
        if col == 'product_type_name':
            threshold_percentage = 0.02
        if col == 'product_group_name':
            threshold_percentage = 0.05
        if col == 'graphical_appearance_name':
            threshold_percentage = 0.02
        if col == 'index_group_name':
            threshold_percentage = 0.05
        counts = df[col].value_counts()
        threshold_count = total_rows * threshold_percentage
        rare_categories = counts[counts < threshold_count].index
        df[col] = df[col].apply(lambda x: 'other' if x in rare_categories else x)
    return df

processed_articles = replace_rare_categories(selected_columns)
processed_articles.head()

<ipython-input-13-541516a8bfdc>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].apply(lambda x: 'other' if x in rare_categories else x)
<ipython-input-13-541516a8bfdc>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].apply(lambda x: 'other' if x in rare_categories else x)
<ipython-input-13-541516a8bfdc>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats

,article_id,product_type_name,product_group_name,graphical_appearance_name,index_group_name
0,108775015,Vest top,Garment Upper body,Solid,Ladieswear
1,108775044,Vest top,Garment Upper body,Solid,Ladieswear
2,108775051,Vest top,Garment Upper body,Stripe,Ladieswear
3,110065001,Bra,Underwear,Solid,Ladieswear
4,110065002,Bra,Underwear,Solid,Ladieswear


In [14]:
from sklearn.preprocessing import OneHotEncoder

# Select categorical columns for one-hot encoding
categorical_cols = ['product_type_name', 'product_group_name', 'graphical_appearance_name', 'index_group_name']

# Initialize and fit the OneHotEncoder
one_hot_encoder = OneHotEncoder(sparse=False)
encoded_cols = one_hot_encoder.fit_transform(processed_articles[categorical_cols])

# Create a DataFrame with the encoded columns and feature names
encoded_df = pd.DataFrame(
    encoded_cols,
    columns=[f"{col}_{val}" for col, vals in zip(categorical_cols, one_hot_encoder.categories_) for val in vals]
)

# Concatenate the encoded DataFrame with the original DataFrame and reset the index to article_id
processed_articles = pd.concat([processed_articles.drop(columns=categorical_cols), encoded_df], axis=1)
processed_articles.set_index('article_id', inplace=True)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [15]:
processed_articles.head()

,product_type_name_Blouse,product_type_name_Bra,product_type_name_Dress,product_type_name_Hoodie,product_type_name_Jacket,product_type_name_Shirt,product_type_name_Shorts,product_type_name_Skirt,product_type_name_Sweater,product_type_name_T-shirt,...,graphical_appearance_name_Melange,graphical_appearance_name_Placement print,graphical_appearance_name_Solid,graphical_appearance_name_Stripe,graphical_appearance_name_other,index_group_name_Baby/Children,index_group_name_Divided,index_group_name_Ladieswear,index_group_name_Menswear,index_group_name_other
article_id,,,,,,,,,,,,,,,,,,,,,
108775015,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
108775044,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
108775051,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
110065001,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
110065002,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [16]:
"""
# Checking the amount of "other" in each column
for col in processed_articles.columns:
    print(f"Value counts for column '{col}':")
    print(processed_articles[col].value_counts())
    print()
"""

'\n# Checking the amount of "other" in each column\nfor col in processed_articles.columns:\n    print(f"Value counts for column \'{col}\':")\n    print(processed_articles[col].value_counts())\n    print()\n'

In [18]:
# Use only transactions of the products on the online site and they do not have duplications in the description
online_data = transactions_df[(transactions_df['sales_channel_id'] == 2) &
                              (transactions_df['article_id'].isin(product_id_list)) &
                              (transactions_df['article_id'].isin(product_id_list2))]

# Grouping by customer - all products purchased, we only took customers who have more than 10 purchases of unique products (or equal to 10 for the example)
grouped_data = online_data.groupby('customer_id')['article_id'].apply(lambda x: list(x.unique())).reset_index()
grouped_data.columns = ['customer_id', 'unique_article_ids']
grouped_data = grouped_data[grouped_data['unique_article_ids'].apply(lambda x: 10 == len(x))]

# Display the grouped data
grouped_data.head()

,customer_id,unique_article_ids
38,0002e6cdaab622b5047407efc0d0bf85e23220e0920120...,"[647946004, 649046002, 693642001, 678625001, 8..."
54,0003e867a930d0d6842f923d6ba7c9b77aba33fe2a0fbf...,"[823025001, 377277001, 870962001, 733749001, 8..."
56,00040239317e877c77ac6e79df42eb2633ad38fcac09fc...,"[782073001, 722169001, 765926001, 220094001, 7..."
67,0004ee708acad44b15ee324daffef7b379a1fdfd9826f3...,"[697491001, 674020001, 763270001, 656868001, 7..."
77,00058c11019467570a6918669b768392f7c9277b7ae4d8...,"[576050001, 620625001, 645065001, 640174001, 6..."


In [19]:
# We took the transactions only of the online products and they are in the embedding description and they do not have duplicates in the description
online_data = transactions_df[(transactions_df['sales_channel_id'] == 2) &
                              (transactions_df['article_id'].isin(product_id_list)) &
                              (transactions_df['article_id'].isin(product_id_list2))]

In [20]:
# Grouping by customer - all the products he purchased, we only took as an example customers who have 10 products in total
grouped_data = online_data.groupby('customer_id')['article_id'].apply(lambda x: list(x.unique())).reset_index()

# Rename columns
grouped_data.columns = ['customer_id', 'unique_article_ids']
grouped_data = grouped_data[grouped_data['unique_article_ids'].apply(lambda x: 10 == len(x))]

# Display the grouped data
grouped_data.head()

,customer_id,unique_article_ids
38,0002e6cdaab622b5047407efc0d0bf85e23220e0920120...,"[647946004, 649046002, 693642001, 678625001, 8..."
54,0003e867a930d0d6842f923d6ba7c9b77aba33fe2a0fbf...,"[823025001, 377277001, 870962001, 733749001, 8..."
56,00040239317e877c77ac6e79df42eb2633ad38fcac09fc...,"[782073001, 722169001, 765926001, 220094001, 7..."
67,0004ee708acad44b15ee324daffef7b379a1fdfd9826f3...,"[697491001, 674020001, 763270001, 656868001, 7..."
77,00058c11019467570a6918669b768392f7c9277b7ae4d8...,"[576050001, 620625001, 645065001, 640174001, 6..."


In [21]:
# Explode the grouped_data to have one row per customer-article pair
exploded_data = grouped_data.explode('unique_article_ids')

# Group by article_id and count the number of unique customers
article_counts = exploded_data.groupby('unique_article_ids')['customer_id'].nunique().reset_index()


# Find an article_id that is purchased by exactly X unique customers - range of 50 to 100 for example
lst = []
for i in range(50,100):
  article_with_x_customers = article_counts[article_counts['customer_id'] == i]['unique_article_ids']
  lst.extend(article_with_x_customers)

# To run the code on hundreds of products - we put all the commands into a function and ran in a loop on the list we selected, In the rest of the notebook, the code appears individually for one product - not within a function

In [22]:
"""
def func(temp_article):
    # Filter customers who have purchased the specific article (temp_article)
    with_article = exploded_data[exploded_data['unique_article_ids'] == temp_article]

    # Randomly sample 50 customers who have purchased the temp_article
    sample_with_article = with_article['customer_id'].sample(n=50, random_state=42)
    sample_with_article = list(sample_with_article)

    # Filter customers who have not purchased the specific article (temp_article)
    without_article = exploded_data[exploded_data['unique_article_ids'] != temp_article]

    # Randomly sample 50 customers who have not purchased the temp_article
    sample_without_article = without_article['customer_id'].sample(n=50, random_state=42)
    sample_without_article = list(sample_without_article)

    # Combine the two samples to get a list of 100 customer IDs
    customers_100_list = sample_with_article + sample_without_article

    # Filter the grouped data to include only the selected 100 customers
    filtered_grouped_data = grouped_data[grouped_data['customer_id'].isin(customers_100_list)]

    # Extract unique article IDs from the filtered grouped data
    unique_articles = set([article_id for sublist in filtered_grouped_data['unique_article_ids'] for article_id in sublist])
    unique_article_list = list(unique_articles)

    # Remove the temp_article from the unique articles list
    unique_article_list.remove(temp_article)

  filtered_description_emb = description_emb[description_emb.index.isin(unique_article_list)]
  filtered_description_emb2 = description_emb[description_emb.index.isin(list(test_article_id))]
  filtered_images_df = images_df[images_df.index.isin(unique_article_list)]
  filtered_images_df2 = images_df[images_df.index.isin(test_article_id)]
  filtered_processed_articles = processed_articles[processed_articles.index.isin(unique_article_list)]
  filtered_processed_articles2 = processed_articles[processed_articles.index.isin(test_article_id)]

  test_art = test_article_id[0]

  # Filter the DataFrame
  filtered_df = network_df[(network_df['a'] == test_art) | (network_df['b'] == test_art)]

  # Sort the DataFrame by the 'cs' column in descending order
  sorted_df = filtered_df.sort_values(by='cs', ascending=False)

  # List to store the results
  result = []

  # Collect the first 5 unique numbers that are not test_art from columns 'a' or 'b'
  for index, row in sorted_df.iterrows():
      if len(result) >= 5:
          break
      if row['a'] != test_art and row['a'] not in result:
          result.append(row['a'])
      if row['b'] != test_art and row['b'] not in result:
          result.append(row['b'])

  # Ensure the list is only 5 elements long
  result = result[:5]

  # Group by article and sum the counts
  result_df = item_count.copy()

  def get_top_2_articles(article_number, network_df, result_df):
    # Filter rows where the input article is present in either column 'a' or 'b'
    filtered_rows = network_df[(network_df['a'] == article_number) | (network_df['b'] == article_number)]

    norm_counts = []
    for _, row in filtered_rows.iterrows():
        # Determine the article paired with the input article in the current row
        if row['a'] == article_number:
            other_article = row['b']
        else:
            other_article = row['a']

        # Find the total count of the other article from the result_df
        total_count_other_article = result_df[result_df['article'] == other_article]['count'].values[0]

        # Calculate the normalized count for the pair
        norm_count = row['count'] / total_count_other_article
        norm_counts.append((other_article, norm_count))

    # Sort the articles by their normalized counts in descending order
    norm_counts.sort(key=lambda x: x[1], reverse=True)

    # Extract the top 2 associated articles
    top_2_articles = [article for article, _ in norm_counts[:2]]

    return top_2_articles

  unique_article_list = []
  for article in result:
    unique_article_list.extend(get_top_2_articles(article, network_df, result_df))

  filtered_description_emb2_avg_vector = pd.DataFrame(filtered_description_emb_for_avg_vector.mean(axis=0), columns=['Average'])


  filtered_description_emb_for_avg_vector = description_emb[description_emb.index.isin(unique_article_list)]

  filtered_description_emb2_avg_vector = pd.DataFrame(filtered_description_emb_for_avg_vector.mean(axis=0), columns=['Average'])
  filtered_description_emb2_avg_vector = filtered_description_emb2_avg_vector.transpose()


  filtered_images_df_for_avg_vector = images_df[images_df.index.isin(unique_article_list)]

  filtered_images_df2_avg_vector = pd.DataFrame(filtered_images_df_for_avg_vector.mean(axis=0), columns=['Average'])
  filtered_images_df2_avg_vector = filtered_images_df2_avg_vector.transpose()


  filtered_processed_articles_for_avg_vector = processed_articles[processed_articles.index.isin(unique_article_list)]

  filtered_processed_articles2_avg_vector = pd.DataFrame(filtered_processed_articles_for_avg_vector.mean(axis=0), columns=['Average'])
  filtered_processed_articles2_avg_vector = filtered_processed_articles2_avg_vector.transpose()

  # Calculate cosine similarity
  cosine_similarities = cosine_similarity(filtered_description_emb.values,
                                          filtered_description_emb_for_avg_vector.values)

  # Calculate the average cosine similarity for each product in filtered_description_emb
  avg_cosine_similarities = cosine_similarities.mean(axis=1)

  # Create a DataFrame to store the product_id and their average cosine similarities
  avg_similarity_df_desc = pd.DataFrame({
      'product_id': filtered_description_emb.index,
      'average_cosine_similarity': avg_cosine_similarities
  })

  avg_similarity_df_desc.set_index('product_id', inplace=True)

  #### scaler
  min_value = avg_similarity_df_desc.min()
  max_value = avg_similarity_df_desc.max()
  avg_similarity_df_desc = (avg_similarity_df_desc - min_value) / (max_value - min_value)
  ####


  # Calculate cosine similarity
  cosine_similarities = cosine_similarity(filtered_images_df.values,
                                          filtered_images_df_for_avg_vector.values)

  # Calculate the average cosine similarity for each product in filtered_description_emb
  avg_cosine_similarities = cosine_similarities.mean(axis=1)

  # Create a DataFrame to store the product_id and their average cosine similarities
  avg_similarity_df_image = pd.DataFrame({
      'product_id': filtered_images_df.index,
      'average_cosine_similarity': avg_cosine_similarities
  })

  avg_similarity_df_image.set_index('product_id', inplace=True)

  #### scaler
  min_value = avg_similarity_df_image.min()
  max_value = avg_similarity_df_image.max()
  avg_similarity_df_image = (avg_similarity_df_image - min_value) / (max_value - min_value)
  ####


  # Calculate cosine similarity
  cosine_similarities = cosine_similarity(filtered_processed_articles.values,
                                          filtered_processed_articles_for_avg_vector.values)

  # Calculate the average cosine similarity for each product in filtered_description_emb
  avg_cosine_similarities = cosine_similarities.mean(axis=1)

  # Create a DataFrame to store the product_id and their average cosine similarities
  avg_similarity_df_tabular = pd.DataFrame({
      'product_id': filtered_processed_articles.index,
      'average_cosine_similarity': avg_cosine_similarities
  })

  avg_similarity_df_tabular.set_index('product_id', inplace=True)

  #### scaler
  min_value = avg_similarity_df_tabular.min()
  max_value = avg_similarity_df_tabular.max()
  avg_similarity_df_tabular = (avg_similarity_df_tabular - min_value) / (max_value - min_value)
  ####


  from google.colab import drive

  # Mount Google Drive
  drive.mount('/content/gdrive')

  # Define the path to save the CSV file
  path = '/content/gdrive/MyDrive/'

  number = str(test_art)+"_"
  filtered_description_emb.to_csv( path + number + 'filtered_description_emb.csv', index=True)
  filtered_description_emb2.to_csv( path + number + 'filtered_description_emb2.csv', index=True)
  filtered_description_emb2_avg_vector.to_csv( path + number + 'filtered_description_emb2_avg_vector.csv', index=True)
  avg_similarity_df_desc.to_csv( path + number + 'filtered_description_emb2_avg_vector_include_cs.csv', index=True)

  filtered_images_df.to_csv( path + number + 'filtered_images_df.csv', index=True)
  filtered_images_df2.to_csv( path + number + 'filtered_images_df2.csv', index=True)
  filtered_images_df2_avg_vector.to_csv( path + number + 'filtered_images_df2_avg_vector.csv', index=True)
  avg_similarity_df_image.to_csv( path + number + 'filtered_images_df2_avg_vector_include_cs.csv', index=True)

  filtered_processed_articles.to_csv( path + number + 'filtered_processed_articles.csv', index=True)
  filtered_processed_articles2.to_csv( path + number + 'filtered_processed_articles2.csv', index=True)
  filtered_processed_articles2_avg_vector.to_csv( path + number + 'filtered_processed_articles2_avg_vector.csv', index=True)
  avg_similarity_df_tabular.to_csv( path + number + 'filtered_processed_articles2_avg_vector_include_cs.csv', index=True)

  filtered_grouped_data.to_csv( path + number + 'filtered_grouped_data.csv', index=True)
"""

'\ndef func(temp_article):\n    # Filter customers who have purchased the specific article (temp_article)\n    with_article = exploded_data[exploded_data[\'unique_article_ids\'] == temp_article]\n\n    # Randomly sample 50 customers who have purchased the temp_article\n    sample_with_article = with_article[\'customer_id\'].sample(n=50, random_state=42)\n    sample_with_article = list(sample_with_article)\n\n    # Filter customers who have not purchased the specific article (temp_article)\n    without_article = exploded_data[exploded_data[\'unique_article_ids\'] != temp_article]\n\n    # Randomly sample 50 customers who have not purchased the temp_article\n    sample_without_article = without_article[\'customer_id\'].sample(n=50, random_state=42)\n    sample_without_article = list(sample_without_article)\n\n    # Combine the two samples to get a list of 100 customer IDs\n    customers_100_list = sample_with_article + sample_without_article\n    \n    # Filter the grouped data to includ

In [23]:
"""
count = 0
for article in lst:
  func(article)
  count+=1
  print(count,article)
"""

'\ncount = 0\nfor article in lst:\n  func(article)\n  count+=1\n  print(count,article)\n'

# Below is the continuation of the code for one product

In [24]:
temp_article = lst[0] # for example

# Filter customers who have the specific article
with_article = exploded_data[exploded_data['unique_article_ids'] == temp_article]

# Sample 50 customer IDs from the subset where the article is present
sample_with_article = with_article['customer_id'].sample(n=50, random_state=42)
sample_with_article = list(sample_with_article)

# Filter customers who do not have the specific article
without_article = exploded_data[exploded_data['unique_article_ids'] != temp_article]

# Sample 50 customer IDs from the subset where the article is absent
sample_without_article = without_article['customer_id'].sample(n=50, random_state=42)
sample_without_article = list(sample_without_article)


In [25]:
# Check - 100 customers (50-50)
customers_100_list = sample_with_article + sample_without_article
print (len(set(customers_100_list)))

100


In [26]:
# We removed only the 100 customers and the products they purchased from the entire data
filtered_grouped_data = grouped_data[grouped_data['customer_id'].isin(customers_100_list)]
filtered_grouped_data.head()

,customer_id,unique_article_ids
31375,08c70c3f1852e81c02a3a347d930fa6c484545c20c9526...,"[637515001, 680810001, 655855001, 619580001, 6..."
53381,0eeff573646b3ed2fffe456dba10ce07a3221b3010cd00...,"[701853001, 719957001, 719655001, 469658009, 6..."
74546,14d1d56bfe8cb08388bcc755cb3b1286f62b3a7282f316...,"[598859001, 760975001, 773955001, 745250001, 7..."
85794,17f3d41c172f15158d79990f83358be801b09e571a50c7...,"[598859001, 821031002, 791215001, 820988001, 8..."
103398,1cd61e8178d47b47fc0eae0af232c8581870ad73307d5c...,"[717341001, 598859001, 706404001, 718292001, 6..."


In [27]:
# We took the products bought by the 100 customers and put them in a list of unique products

# Flatten the lists of article IDs and create a set of unique article IDs
unique_articles = set([article_id for sublist in filtered_grouped_data['unique_article_ids'] for article_id in sublist])

# Convert the set back to a list (if a list is preferred)
unique_article_list = list(unique_articles)

print(len(unique_article_list))

859


In [28]:
# Removal of the test product from the data
unique_article_list.remove(temp_article)
test_article_id = [temp_article]

# Preparing the data for the model - a training set (all products purchased by the 100 customers), a test set (the test product), in 3 dimensions

In [29]:
# Train - description
filtered_description_emb = description_emb[description_emb.index.isin(unique_article_list)]
filtered_description_emb

,0,1,2,3,4,5,6,7,8,9,...,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023
product_id,,,,,,,,,,,,,,,,,,,,,
108775015,0.643704,-0.728313,-0.370916,0.930039,0.205670,-0.512119,-0.781456,0.846769,0.224285,0.927329,...,0.184097,-0.499265,0.154369,0.201446,-1.428334,1.050288,-0.188325,0.233521,-0.609104,0.525521
146730001,-0.559831,0.160561,-0.449470,0.885812,-0.125695,-0.130094,-0.770814,0.699748,0.098937,-0.218613,...,0.489990,0.759986,-1.217662,-0.034277,-0.357784,1.521733,-0.410435,-0.311910,0.480676,-0.337573
179123001,0.458272,0.059285,0.276166,1.059452,0.443960,-0.288570,-1.487896,1.692867,0.706955,0.634377,...,0.236120,0.426178,-0.443457,0.405998,-1.029748,0.597395,-0.110130,0.535507,-0.493158,-0.352034
188183001,-1.070638,0.125105,-0.550470,-0.314886,1.344844,-0.122543,-0.467595,0.774600,-0.415915,1.714809,...,-0.890185,0.698115,-1.129992,0.049247,0.584341,0.319822,-0.608848,1.214177,-1.859561,0.070931
189626001,0.480388,-0.418167,0.061974,0.260839,0.078575,-0.407894,-0.245582,1.059476,-0.049720,0.885928,...,-1.304751,0.108009,-1.113686,0.351742,-0.023073,0.890193,0.896355,0.235009,-0.570083,-0.727708
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
919829001,-0.194801,-0.120765,-0.938388,0.800233,0.577251,-0.868372,-0.294473,-0.413461,-0.104212,-0.546273,...,1.120060,0.772355,-0.926850,-0.207181,-0.696452,0.083883,-0.907433,0.778877,-0.213863,0.333119
920972001,-0.436156,0.443664,-1.072007,0.525909,0.882940,-1.023450,-0.847799,-0.675804,-0.539391,0.922721,...,-0.067730,0.494777,-0.325209,1.343376,0.143810,1.403391,-0.483553,0.521945,-1.266280,-0.344269
924625001,0.061793,-0.675045,-0.637907,0.209245,0.637579,1.095584,-0.241692,-0.092321,0.608598,0.716013,...,0.613691,0.839969,-0.631349,0.735252,-0.061283,-0.517386,0.379356,0.554224,0.008105,-0.415060


In [30]:
# Test - description
filtered_description_emb2 = description_emb[description_emb.index.isin(list(test_article_id))]
filtered_description_emb2

,0,1,2,3,4,5,6,7,8,9,...,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023
product_id,,,,,,,,,,,,,,,,,,,,,
598859001,-0.405513,-0.051569,-0.989036,-0.60737,0.045394,-0.087561,0.038358,-0.345562,0.225971,-0.060393,...,-0.799842,-0.604129,-0.845667,0.771863,1.223258,0.022761,-0.409028,0.773971,-1.287718,0.257793


In [31]:
# Train - images
filtered_images_df = images_df[images_df.index.isin(unique_article_list)]
filtered_images_df

,0,1,2,3,4,5,6,7,8,9,...,502,503,504,505,506,507,508,509,510,511
product_id,,,,,,,,,,,,,,,,,,,,,
640639001,0.002164,0.160031,0.077019,1.318911,0.082665,0.922043,0.453130,0.292028,1.633524,0.147706,...,0.438292,0.091234,2.042429,1.442743,1.959560,0.154487,0.135294,1.268311,0.433102,0.856287
573937001,0.216983,0.096842,0.923034,1.319407,0.022791,0.397418,0.441368,0.368506,0.712484,0.326254,...,0.090512,0.170772,1.506806,0.902479,0.053314,0.257971,0.568578,0.214248,0.203147,0.326643
611415001,1.406232,0.546891,0.580072,1.120027,0.742537,1.129162,0.111230,0.935790,1.645105,0.697254,...,1.868597,1.279962,1.584500,1.389971,0.554970,0.082156,1.176609,0.267532,0.795614,0.671178
672590001,0.396448,0.095949,0.030547,0.651544,0.018536,3.010272,0.172714,0.170215,2.799231,0.128462,...,0.114843,0.928319,3.395531,0.394646,0.228162,0.028572,1.216061,1.193800,1.316798,0.005410
632143001,0.611340,0.437140,0.000000,1.851085,0.000000,2.095433,0.538131,0.258265,3.703549,0.388144,...,0.236166,0.543128,3.859542,1.827221,0.143815,0.911038,0.861525,0.799896,0.110939,0.273456
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
902163001,1.508715,1.571242,0.068154,0.783916,0.038777,2.916176,0.184703,0.127756,4.438637,0.063962,...,0.283308,0.037539,1.649868,0.604601,0.213592,0.586368,1.581801,1.660544,0.910847,0.228621
918171001,0.256382,0.624202,0.031853,0.234001,0.055222,1.455386,0.109994,0.267852,2.780632,0.260458,...,0.118923,0.392911,1.197570,0.287470,0.495583,0.936115,0.932997,1.434020,0.327073,0.038838
919829001,1.178251,0.742505,0.001508,1.241291,0.185292,2.278036,0.092347,0.005376,2.511010,0.828002,...,0.627095,0.005731,4.511308,1.753014,1.117818,0.219645,0.757779,0.570077,0.462277,0.415257


In [32]:
# Test - images
filtered_images_df2 = images_df[images_df.index.isin(test_article_id)]
filtered_images_df2

,0,1,2,3,4,5,6,7,8,9,...,502,503,504,505,506,507,508,509,510,511
product_id,,,,,,,,,,,,,,,,,,,,,
598859001,1.437648,0.133718,0.037859,0.41299,0.0,1.274141,0.062468,0.142024,3.199175,0.52181,...,0.021453,0.238905,2.746033,0.436664,0.392804,0.82152,0.998778,1.007578,1.293299,0.008359


In [33]:
# Train - Tabular
filtered_processed_articles = processed_articles[processed_articles.index.isin(unique_article_list)]
filtered_processed_articles

,product_type_name_Blouse,product_type_name_Bra,product_type_name_Dress,product_type_name_Hoodie,product_type_name_Jacket,product_type_name_Shirt,product_type_name_Shorts,product_type_name_Skirt,product_type_name_Sweater,product_type_name_T-shirt,...,graphical_appearance_name_Melange,graphical_appearance_name_Placement print,graphical_appearance_name_Solid,graphical_appearance_name_Stripe,graphical_appearance_name_other,index_group_name_Baby/Children,index_group_name_Divided,index_group_name_Ladieswear,index_group_name_Menswear,index_group_name_other
article_id,,,,,,,,,,,,,,,,,,,,,
108775015,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
146730001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
179123001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
188183001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
189626001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
919829001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
920972001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
924625001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [34]:
# Test - Tabular
filtered_processed_articles2 = processed_articles[processed_articles.index.isin(test_article_id)]
filtered_processed_articles2

,product_type_name_Blouse,product_type_name_Bra,product_type_name_Dress,product_type_name_Hoodie,product_type_name_Jacket,product_type_name_Shirt,product_type_name_Shorts,product_type_name_Skirt,product_type_name_Sweater,product_type_name_T-shirt,...,graphical_appearance_name_Melange,graphical_appearance_name_Placement print,graphical_appearance_name_Solid,graphical_appearance_name_Stripe,graphical_appearance_name_other,index_group_name_Baby/Children,index_group_name_Divided,index_group_name_Ladieswear,index_group_name_Menswear,index_group_name_other
article_id,,,,,,,,,,,,,,,,,,,,,
598859001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


# Calculating the average vector - finding the 10 most bought products with the 5 products most similar to the test vector

In [35]:
test_art = test_article_id[0]
# Filter the DataFrame
filtered_df = network_df[(network_df['a'] == test_art) | (network_df['b'] == test_art)]

# Sort the DataFrame by the 'cs' column in descending order
sorted_df = filtered_df.sort_values(by='cs', ascending=False)

# List to store the results
result = []

# Collect the first 5 unique numbers that are not test_art from columns 'a' or 'b'
for index, row in sorted_df.iterrows():
    if len(result) >= 5:
        break
    if row['a'] != test_art and row['a'] not in result:
        result.append(row['a'])
    if row['b'] != test_art and row['b'] not in result:
        result.append(row['b'])

# Ensure the list is only 5 elements long
result = result[:5]

result

[619825001.0, 777737001.0, 543729003.0, 598859009.0, 699262001.0]

In [36]:
# Group by article and sum the counts
result_df = item_count.copy()

def get_top_2_articles(article_number, network_df, result_df):
    # Filter rows where the article is in column 'a' or 'b'
    filtered_rows = network_df[(network_df['a'] == article_number) | (network_df['b'] == article_number)]

    norm_counts = []

    for _, row in filtered_rows.iterrows():
        if row['a'] == article_number:
            other_article = row['b']
        else:
            other_article = row['a']

        total_count_other_article = result_df[result_df['article'] == other_article]['count'].values[0]
        norm_count = row['count'] / total_count_other_article
        norm_counts.append((other_article, norm_count))

    norm_counts.sort(key=lambda x: x[1], reverse=True)
    top_2_articles = [article for article, _ in norm_counts[:2]]
    return top_2_articles


In [37]:
unique_article_list = []
for article in result:
  unique_article_list.extend(get_top_2_articles(article, network_df, result_df))

print(unique_article_list)

[639794001.0, 489512001.0, 777737002.0, 822376001.0, 677447001.0, 638192003.0, 625114023.0, 723147004.0, 703467001.0, 699263001.0]


In [38]:
# Calculation of the mean vector - description
filtered_description_emb_for_avg_vector = description_emb[description_emb.index.isin(unique_article_list)]
filtered_description_emb2_avg_vector = pd.DataFrame(filtered_description_emb_for_avg_vector.mean(axis=0), columns=['Average'])
filtered_description_emb2_avg_vector = filtered_description_emb2_avg_vector.transpose()
filtered_description_emb2_avg_vector

,0,1,2,3,4,5,6,7,8,9,...,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023
Average,0.01416,-0.151538,-0.718852,0.071476,0.443172,-0.581928,-0.031351,-0.410598,0.183482,-0.090367,...,0.099152,0.073205,-0.688908,0.21727,0.19769,0.836135,-0.019422,0.800764,-0.667036,-0.310796


In [39]:
# Calculation of the mean vector - images
filtered_images_df_for_avg_vector = images_df[images_df.index.isin(unique_article_list)]
filtered_images_df2_avg_vector = pd.DataFrame(filtered_images_df_for_avg_vector.mean(axis=0), columns=['Average'])
filtered_images_df2_avg_vector = filtered_images_df2_avg_vector.transpose()
filtered_images_df2_avg_vector

,0,1,2,3,4,5,6,7,8,9,...,502,503,504,505,506,507,508,509,510,511
Average,1.199681,0.719886,0.106546,0.954086,0.154331,1.74329,0.267116,0.351151,1.96879,0.573528,...,0.598278,0.653701,2.141201,1.258089,0.844735,0.476605,1.256576,0.862592,0.393772,0.994271


In [40]:
# Calculation of the mean vector - Tabular
filtered_processed_articles_for_avg_vector = processed_articles[processed_articles.index.isin(unique_article_list)]
filtered_processed_articles2_avg_vector = pd.DataFrame(filtered_processed_articles_for_avg_vector.mean(axis=0), columns=['Average'])
filtered_processed_articles2_avg_vector = filtered_processed_articles2_avg_vector.transpose()
filtered_processed_articles2_avg_vector

,product_type_name_Blouse,product_type_name_Bra,product_type_name_Dress,product_type_name_Hoodie,product_type_name_Jacket,product_type_name_Shirt,product_type_name_Shorts,product_type_name_Skirt,product_type_name_Sweater,product_type_name_T-shirt,...,graphical_appearance_name_Melange,graphical_appearance_name_Placement print,graphical_appearance_name_Solid,graphical_appearance_name_Stripe,graphical_appearance_name_other,index_group_name_Baby/Children,index_group_name_Divided,index_group_name_Ladieswear,index_group_name_Menswear,index_group_name_other
Average,0.0,0.0,0.0,0.1,0.2,0.1,0.0,0.0,0.1,0.0,...,0.0,0.0,0.6,0.0,0.1,0.3,0.1,0.4,0.2,0.0


# To run in parallel, the code here generates the required files and exports them:
# 1. The product data of the 100 customers in the 3 dimensions
# 2. Test product data in 3 dimensions
# 3. The average vector data in 3 dimensions
# 4. The data of the customers who purchased the test product

In [41]:

from google.colab import drive

# Mount Google Drive
drive.mount('/content/gdrive')

# Define the path to save the CSV file
path = '/content/gdrive/MyDrive/'

number = str(test_art)+"_"

# The product data of the 100 customers in the 3 dimensions
filtered_description_emb.to_csv( path + number + 'filtered_description_emb.csv', index=True)
filtered_images_df.to_csv( path + number + 'filtered_images_df.csv', index=True)
filtered_processed_articles.to_csv( path + number + 'filtered_processed_articles.csv', index=True)

# Test product data in 3 dimensions
filtered_description_emb2.to_csv( path + number + 'filtered_description_emb2.csv', index=True)
filtered_images_df2.to_csv( path + number + 'filtered_images_df2.csv', index=True)
filtered_processed_articles2.to_csv( path + number + 'filtered_processed_articles2.csv', index=True)

# The average vector data in 3 dimensions
filtered_description_emb2_avg_vector.to_csv( path + number + 'filtered_description_emb2_avg_vector.csv', index=True)
filtered_images_df2_avg_vector.to_csv( path + number + 'filtered_images_df2_avg_vector.csv', index=True)
filtered_processed_articles2_avg_vector.to_csv( path + number + 'filtered_processed_articles2_avg_vector.csv', index=True)

# The data of the customers who purchased the test product
filtered_grouped_data.to_csv( path + number + 'filtered_grouped_data.csv', index=True)


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
